In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=d8ccc62f8eb0cf28f5623480ae98e4166ddc4772ff791a05dbb1881a0026658d
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F10 '''
def F10(X):
    f = bn.F10()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 2522.4 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_10_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_10_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0,10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=291354, Fri Apr 17 12:23:29 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.295027836470197e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.200392038577099e+08 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.49664711e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.27227543e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.090551140013684e+08 1.0e+00 2.35e+00  2e+00  2e+00 0:00.3
   28    532 3.495287033676662e+07 1.1e+00 2.05e+00  2e+00  2e+00 0:03.4
   62   1178 2.597916927430004e+06 1.1e+00 1.63e+00  2e+00  2e+00 0:07.4
  100   1900 -4.992804998088691e+07 1.2e+00 1.16e+00  1e+00  1e+00 0:12.0
  153   2907 -1.743793938829842e+08 1.2e+00 6.78e-01  7e-01  7e-01 0:18.1
  200   3800 -2.261679740838594e+08 1.2e+00 4.07e-01  4e-01  4e-01 0:23.5
  269   5111 -2.510318036843971e+08 1.2e+00 1.80e-01  2e-01  2e-01 0:31.6
  300   5700 -2.553241394556352e+08 1.2e+00 1.23e-01  1e-01  1e-01 0:35.2
  385   7315 -2.581069260411959e+08 1.2e+00 4.41e-02  4e-02  4e-02 0:45.2
  400   7600 -2.582452735723083e+08 1.2e+00 3.76e-02  4e-02  4e-02 0:47.0
  500   9500 -2.585030494016967e+08 1.2e+00 1.27e-02  1e-02  1e-02 0:58.8
  600  11400 -2.585353467276200e+08 1.2e+00 4.36e-03  4e-03  4e-03 1:10.8
  700  13300 -2.585385892079351e+08 1.2e+00 1.43e-03  1e-03  1e-03 1:22.9
  800  15200 -2.585390571444660e+08 1.2e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.56875172e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=156875172.076549, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=156875172.076549, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Fri Apr 17 12:46:12 2020)
final/bestever f-value = 1.568752e+08 1.568752e+08
incumbent solution: [ 2.2261833   0.71842875  2.36124667 -2.62138295  3.22842374 -3.01534343
 -3.15644223  0.20847228 ...]
std deviations: [2.43747624 2.43766289 2.43710664 2.43677217 2.43764513 2.43745597
 2.43753886 2.43761396 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=234584, Fri Apr 17 12:46:12 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.079724582899621e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 7.059459758043352e+07 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


...

 9100 172900 -5.966030780368357e+08 9.6e+00 1.38e-08  1e-08  8e-08 17:01.3
 9200 174800 -5.966030780368358e+08 9.7e+00 1.22e-08  8e-09  7e-08 17:12.6
 9300 176700 -5.966030780368359e+08 9.6e+00 1.10e-08  8e-09  7e-08 17:23.8
 9400 178600 -5.966030780368359e+08 9.7e+00 9.15e-09  6e-09  5e-08 17:35.1
 9500 180500 -5.966030780368359e+08 9.7e+00 7.16e-09  5e-09  4e-08 17:46.3
 9600 182400 -5.966030780368359e+08 9.7e+00 5.96e-09  4e-09  4e-08 17:57.6


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-596603078.036836, sigma=5.04e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=9652)
  ')')


 9700 184300 -5.966030780368359e+08 9.7e+00 4.74e-09  3e-09  3e-08 18:08.8
 9800 186200 -5.966030780368359e+08 9.8e+00 4.62e-09  3e-09  3e-08 18:20.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-596603078.036836, sigma=4.84e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=9888)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-596603078.036836, sigma=4.75e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=9895)
  ')')


 9900 188100 -5.966030780368359e+08 9.8e+00 4.70e-09  3e-09  3e-08 18:31.2
10000 190000 -5.966030780368359e+08 9.9e+00 4.58e-09  3e-09  3e-08 18:42.6
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
10100 191900 -5.966030780368359e+08 9.9e+00 4.07e-09  3e-09  2e-08 18:53.9
10200 193800 -5.966030780368359e+08 9.9e+00 4.23e-09  3e-09  3e-08 19:05.0
10300 195700 -5.966030780368359e+08 9.9e+00 4.14e-09  3e-09  2e-08 19:16.2
10326 196194 -5.966030780368359e+08 9.9e+00 4.11e-09  3e-09  2e-08 19:19.2
termination on tolfunhist=1e-12 (Fri Apr 17 19:55:17 2020)
final/bestever f-value = -5.966031e+08 -5.966031e+08
incumbent solution: [ 5.          5.         -5.         -5.          2.39134036  5.
  5.          5.         ...]
std deviations: [3.70769925e-09 2.89780960e-09 6.46730732e-09 3.70746631e-09
 4.45483149e-09 5.78055991e-09 3.50464489e-09 3.84303927e-09 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F10_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F10(X_Values.iloc[i,:200])
    SVM_Fun [i] = F10(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F10(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1.12086623e+08]), count=array([1])) 1.418089701401637
SVM
ModeResult(mode=array([1.55301101e+08]), count=array([1])) 25410636.02939436
ELN
ModeResult(mode=array([1.74625255e+08]), count=array([1])) 40294589.78170942
